In [1]:
from imports import *
from agents import BetaAgent
from model import Model
from network_utils import *
from network_randomization import *
from network_generation import *
from simulation_functions import *

# Testing

In [2]:
n_simulations = 100
G_default = barabasi_albert_directed(100,5)

num_cores = cpu_count()  # Get the number of available CPU cores
print(num_cores)

# Define a partial function to pass G_perceptron to generate_parameters_empir
# This ensures that generate_parameters_empir is called with the correct argument within the pool
# The 'partial' function allows you to create a new function with some of the arguments pre-filled.
from functools import partial
generate_params_with_G = partial(generate_parameters, G=G_default)

with Pool(num_cores) as pool:
    # Use tqdm to display a progress bar
    # Now, 'generate_params_with_G' is the function that will be executed by each worker.
    # Each worker will receive an index from 'range(n_simulations)' as its argument,
    # which is ignored in 'generate_params_with_G' but is required by the 'imap_unordered' function.
    param_dict = list(tqdm.tqdm(pool.imap_unordered(generate_params_with_G, range(n_simulations)), total=n_simulations))

4


100%|██████████| 100/100 [00:03<00:00, 25.02it/s]


In [3]:
print(len(param_dict))
param_dict[0]

100


{'randomized': True,
 'unique_id': 'e1ef1f1fc9234671910e191333c344f8',
 'n_agents': 100,
 'network': <networkx.classes.digraph.DiGraph at 0x1350a7c90>,
 'uncertainty': 0.002452944622785744,
 'n_experiments': 1,
 'p_rewiring': 0.1428470602645185,
 'average_degree': 4.85,
 'degree_gini_coefficient': np.float64(0.7341649484536084),
 'approx_average_clustering_coefficient': 0.1769603529043353,
 'degree_entropy': np.float64(2.123990312382749)}

In [4]:
# Run simulations in parallel
with Pool(num_cores) as pool:
    simulation_results = list(tqdm.tqdm(pool.imap_unordered(run_simulation_wrapper, param_dict),
                                        total=len(param_dict), desc="Running simulations"))

# Convert results to a DataFrame
basic_results_df = pd.DataFrame(simulation_results)
display(basic_results_df)

Running simulations:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
basic_results_df.to_csv("basic_results_df.csv", index=False)  # Saves without index

# Basic Plotting

In [ ]:
rename_dict = {
    'n_agents': 'Number of Agents',
    'p_rewiring': 'Probability of Rewiring',
    'uncertainty': 'Problem Easiness',
    'n_experiments': 'Number of Experiments',
    'share_of_correct_agents_at_convergence': 'Share of correct agents', # is this CORRECT ?!!?!?!?
    #'share_of_correct_agents_at_convergence':'Share of correct agents',
    'mean_degree': 'Mean Degree',
    'ba_degree':'BA-Degree',
    'convergence_step': 'Steps until convergence (log)',
    'approx_average_clustering_coefficient':'Clustering Coeff.',
    'average_degree': 'Average Degree',
    'degree_gini_coefficient': 'Degree Gini Coeff.',
    'diameter': 'Diameter',

}

In [ ]:
X = basic_results_df[['n_agents','mean_degree',	'degree_gini_coefficient'	,'approx_average_clustering_coefficient',
                           'diameter',	'uncertainty',	'n_experiments','p_rewiring','in_entropy','out_entropy']]
y = pd.DataFrame(basic_results_df['share_of_correct_agents_at_convergence'])
X.rename(columns=rename_dict, inplace=True)
y.rename(columns=rename_dict, inplace=True)

plot_scatter_matrix(X, y, max_cols=3, filename='test',hex_colors=['#d9d9d9','#ab0b00'])